In [1]:
from keras.models import load_model
import numpy as np
import pandas as pd
from keras_layer_normalization import LayerNormalization
from keras.models import Model
from sklearn.model_selection import train_test_split

from keras.layers import Input, LSTM, Embedding, Dense

In [2]:
model = load_model('best_model_P.h5')

In [2]:
Urdu=input()

ابھی تو شکر ہے وہ غالب کی بھتیجی ادھر نہیں ہے ہر قصہ اسی طرح دھیرے دھیرے سناتے جس رفتار سے ک


In [23]:
input_seq=[Urdu]

NameError: name 'Urdu' is not defined

In [7]:
lines=pd.read_excel("TotalP.xlsx")

In [8]:
lines.dropna(inplace=True)

In [9]:
lines['Pashto'] = lines['Pashto'].apply(lambda x : 'START_ '+ x + ' _END')

In [10]:
### Get English and Hindi Vocabulary
all_urdu_words=set()
for urdu in lines['Urdu']:
    for word in urdu.split():
        if word not in all_urdu_words:
            all_urdu_words.add(word)

all_Pashto_words=set()
for ps in lines['Pashto']:
    for word in ps.split():
        if word not in all_Pashto_words:
            all_Pashto_words.add(word)

In [11]:
lines['length_urdu_sentence']=lines['Urdu'].apply(lambda x:len(x.split(" ")))
lines['length_sindhi_sentence']=lines['Pashto'].apply(lambda x:len(x.split(" ")))

In [12]:
lines=lines[lines['length_urdu_sentence']<=30]
lines=lines[lines['length_sindhi_sentence']<=30]

In [13]:
input_words = sorted(list(all_urdu_words))
target_words = sorted(list(all_Pashto_words))
num_encoder_tokens = len(all_urdu_words)
num_decoder_tokens = len(all_Pashto_words)
num_encoder_tokens, num_decoder_tokens

(147329, 68139)

In [14]:
num_decoder_tokens += 1 #for zero padding

In [15]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [16]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [17]:
latent_dim=300
max_length_src=30
max_length_tar=30

In [18]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


In [19]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [22]:
def generate_batch(X = input_seq, batch_size = 1):
    ''' Generate a batch of data '''
    now=[']','[',"'"]
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            input_text=str(input_seq)
            i=0
            print(input_text)
            input_text = ''.join(filter(lambda x: x not in now , input_text))
            for t, word in enumerate(input_text.split()):
                encoder_input_data[i, t] = input_token_index[word] # encoder input seq
        yield([encoder_input_data])

NameError: name 'input_seq' is not defined

In [21]:
train_gen = generate_batch(input_seq, batch_size = 1)

NameError: name 'generate_batch' is not defined

In [20]:
(Urdu) = next(train_gen)

['ابھی تو شکر ہے وہ غالب کی بھتیجی ادھر نہیں ہے ہر قصہ اسی طرح دھیرے دھیرے سناتے جس رفتار سے ک']


In [68]:
states_value = encoder_model.predict(Urdu)


In [103]:
df_describe = pd.DataFrame(Urdu)

ValueError: Must pass 2-d input

In [106]:
statesvalues=encoder_model.predict(new)

ValueError: in user code:

    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:1569 run_step  **
        outputs = model.predict_step(data)
 

In [32]:
target_seq = np.zeros((1,1))
target_seq[0, 0] = target_token_index['START_']

In [35]:
stop_condition = False
decoded_sentence = ''
while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + x)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += ' '+sampled_char

    # Exit condition: either hit max length
    # or find stop character.
    if (sampled_char == '_END' or
       len(decoded_sentence) > 50):
        stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # Update states
    states_value = [h, c]

ValueError: in user code:

    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:1569 run_step  **
        outputs = model.predict_step(data)
 

In [23]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [21]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [22]:
prediction=decode_sequence(Urdu)
print(prediction)

 نبری مډډلر Rigigi فایټری ټونډ پلیټوا ثضح سپیکولر سنګاکر
